# Training and Evaluating Time Series Forcasting Model

## Introduction 

In this notebook, we will develop a program to forcast time series data which has seasonal cycles. We will use [NYC Property Sales data](https://www1.nyc.gov/site/finance/about/open-portal.page) range from 2003 to 2015 published by NYC Department of Finance on the [NYC Open Data Portal](https://opendata.cityofnewyork.us/). 

The dataset is a record of every building sold in New York City property market during 13-year period. Please refer to [Glossary of Terms for Property Sales Files](https://www1.nyc.gov/assets/finance/downloads/pdf/07pdf/glossary_rsf071607.pdf) for definition of columns in the spreadsheet. The dataset looks like below:

|borouge|neighborhood|building_class_category|tax_class|block|lot|eastment|building_class_at_present|address|apartment_number|zip_code|residential_units|commercial_units|total_units|land_square_feet|gross_square_feet|year_built|tax_class_at_time_of_sale|building_class_at_time_of_sale|sale_price|sale_date|
|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|
|Manhattan|ALPHABET CITY|07  RENTALS - WALKUP APARTMENTS|0.0|384.0|17.0||C4|225 EAST 2ND   STREET||10009.0|10.0|0.0|10.0|2145.0|6670.0|1900.0|2.0|C4|275000.0|2007-06-19|
|Manhattan|ALPHABET CITY|07  RENTALS - WALKUP APARTMENTS|2.0|405.0|12.0||C7|508 EAST 12TH   STREET||10009.0|28.0|2.0|30.0|3872.0|15428.0|1930.0|2.0|C7|7794005.0|2007-05-21|

We will build up a model to forcast monthly volume of property trade based on history data. In order to forecast, we will use [Facebook Prophet](https://facebook.github.io/prophet/), which provides fast and automated forecast procedure and handles seasonality well.

## Step 1: Load the Data

### Download dataset and Upload to Lakehouse

There are 15 csv files containig property sales records from 5 boroughs in New York since 2003 to 2015. For your convenience, these files are compressed in `nyc_property_sales.tar` and is available in a public blob storage.

In [ ]:
URL = "https://synapseaisolutionsa.blob.core.windows.net/public/NYC_Property_Sales_Dataset/"
TAR_FILE_NAME = "nyc_property_sales.tar"
DATA_FOLER = "Files/NYC_Property_Sales_Dataset"
TAR_FILE_PATH = f"/lakehouse/default/{DATA_FOLER}/tar/"
CSV_FILE_PATH = f"/lakehouse/default/{DATA_FOLER}/csv/"

In [ ]:
import os

if not os.path.exists("/lakehouse/default"):
    # ask user to add a lakehouse if no default lakehouse added to the notebook.
    # a new notebook will not link to any lakehouse by default.
    raise FileNotFoundError(
        "Default lakehouse not found, please add a lakehouse for the notebook."
    )
else:
    # check if the needed files are already in the lakehouse, try to download and unzip if not.
    if not os.path.exists(f"{TAR_FILE_PATH}{TAR_FILE_NAME}"):
        os.makedirs(TAR_FILE_PATH, exist_ok=True)
        os.system(f"wget {URL}{TAR_FILE_NAME} -O {TAR_FILE_PATH}{TAR_FILE_NAME}")

    os.makedirs(CSV_FILE_PATH, exist_ok=True)
    os.system(f"tar -zxvf {TAR_FILE_PATH}{TAR_FILE_NAME} -C {CSV_FILE_PATH}")

### Create Dataframe from Lakehouse

The `display` function print the dataframe and automatically gives chart views.

In [ ]:
df = (
    spark.read.format("csv")
    .option("header", "true")
    .load("Files/NYC_Property_Sales_Dataset/csv")
)
display(df)

## Step 2: Data Preprocessing

### Type Conversion and Filtering
Lets do some necessary type conversion and filtering.
- Need convert sale prices to integers.
- Need exclude irregular sales data. For example, a $0 sale indicate ownership transfer without cash consideration.
- Exclude building types other than A class.

The reason to choose only market of A class building for analysis is that seasonal effect is innegligible coeffient for A class building. The model we will use outperform many others in including seasonality, which is very common needs in time series analysis.

In [ ]:
# import libs
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [ ]:
df = df.withColumn(
    "sale_price", F.regexp_replace("sale_price", "[$,]", "").cast(IntegerType())
)
df = df.select("*").where(
    'sale_price > 0 and total_units > 0 and gross_square_feet > 0 and building_class_at_time_of_sale like "A%"'
)

In [ ]:
monthly_sale_df = df.select(
    "sale_price",
    "total_units",
    "gross_square_feet",
    F.date_format("sale_date", "yyyy-MM").alias("month"),
)

In [ ]:
display(df)

In [ ]:
summary_df = (
    monthly_sale_df.groupBy("month")
    .agg(
        F.sum("sale_price").alias("total_sales"),
        F.sum("total_units").alias("units"),
        F.sum("gross_square_feet").alias("square_feet"),
    )
    .orderBy("month")
)

In [ ]:
display(summary_df)

### Visualization
Now, let's take a look at the trend of property trade trend at NYC. The yearly seasonality is quite clear on the chosen building class. The peek buying seasons are usually spring and fall.

In [ ]:
df_pandas = summary_df.toPandas()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(35, 10))
plt.sca(ax1)
plt.xticks(np.arange(0, 15 * 12, step=12))
plt.ticklabel_format(style="plain", axis="y")
sns.lineplot(x="month", y="total_sales", data=df_pandas)
plt.ylabel("Total Sales")
plt.xlabel("Time")
plt.title("Total Property Sales by Month")

plt.sca(ax2)
plt.xticks(np.arange(0, 15 * 12, step=12))
plt.ticklabel_format(style="plain", axis="y")
sns.lineplot(x="month", y="square_feet", data=df_pandas)
plt.ylabel("Total Square Feet")
plt.xlabel("Time")
plt.title("Total Property Square Feet Sold by Month")
plt.show()

## Step 3: Model Training and Evaluation

### Install Prophet

Now let's install [Facebook Prophet](https://facebook.github.io/prophet/). It uses a decomposable time series model which consist of three main components: trend, seasonality and holidays. 
For the trend part, Prophet assumes piece-wise constant rate of growth with automatic chagne point selection.
For seasonality part, Prophet models weekly and yearly seasonality using Fourier Series. Since we are using monthly data, so we won't have weekly seasonality and won't considering holidays.

In [ ]:
%pip install prophet

### Model Fitting

To do model fitting, we just need to rename the time axis to 'ds' and value axis to 'y'

In [ ]:
import pandas as pd

df_pandas["ds"] = pd.to_datetime(df_pandas["month"])
df_pandas["y"] = df_pandas["total_sales"]

Now let's fit the model. We will choose to use 'multiplicative' seasonality, it means seasonality is no longer a constant additive factor like defualt assumed by Prophet. As you can see a in previous cell, we printed out the total property sale data per month, and the vibration amplitude is not consistant. It means using simple additive seasonlity won't fit the data well.
In addition, we will use Markov Chain Monte Carlo(MCMC) that gives mean of posteriori distribution. By default, Prophet uses Stan's L-BFGS to fit the model, which find a maximum a posteriori probability(MAP) estimate.


In [ ]:
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

m = Prophet(
    seasonality_mode="multiplicative", weekly_seasonality=False, mcmc_samples=1000
)
m.fit(df_pandas)

Let's use bulit-in functions in Prophet to show the model fitting results. The black dots are data points used to train the model. The blue line is the prediction and the light blue area shows uncertainty intervals.

In [ ]:
future = m.make_future_dataframe(periods=12, freq="M")
forecast = m.predict(future)
fig = m.plot(forecast)

Prophet assumes piece-wise constant growth, thus you can plot the change points of the trained model

In [ ]:
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

Visualize trend and yearly seasonality. The light blue area reflects uncertainty.

In [ ]:
fig2 = m.plot_components(forecast)

### Cross Validation

We can use Prophet's built in cross validation functionality to measure the forecast error on historical data. The below parameters means starting with 11 years of training data, then making predictions every 30 days within 1 year horizon.

In [ ]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

df_cv = cross_validation(m, initial="11 Y", period="30 days", horizon="365 days")
df_p = performance_metrics(df_cv, monthly=True)

In [ ]:
display(df_p)